In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import joblib
import os
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\vktaw\Desktop\FashionFinderMyntra\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CSV_PATH = 'Active Product List  Admin 02.04.2025.csv'
MODEL_NAME = 'all-MiniLM-L6-v2'
EMBEDDINGS_PATH = 'product_embeddings.npy'
INDEX_PATH = 'faiss_index.index'
DF_PATH = 'product_df.pkl'

In [3]:
if os.path.exists(EMBEDDINGS_PATH) and os.path.exists(INDEX_PATH) and os.path.exists(DF_PATH):
    df = joblib.load(DF_PATH)
    index = faiss.read_index(INDEX_PATH)
    embeddings = np.load(EMBEDDINGS_PATH)
    model = SentenceTransformer(MODEL_NAME) 
else:
    df = pd.read_csv(CSV_PATH)
    df.fillna('', inplace=True)
    df['search_text'] = df['Name'] + ' ' + df['Type'] + ' ' + df['Category'] + ' ' + df['SubCategory']
    model = SentenceTransformer(MODEL_NAME)
    embeddings = model.encode(df['search_text'].tolist(), show_progress_bar=True)
    np.save(EMBEDDINGS_PATH, embeddings)
    joblib.dump(df, DF_PATH)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    faiss.write_index(index, INDEX_PATH)

In [23]:
def recommend_products(query, threshold=0.5):
    query_embedding = model.encode([query])
    all_embeddings = index.reconstruct_n(0, index.ntotal)
    similarities = cosine_similarity(query_embedding, all_embeddings)[0]
    indices = np.where(similarities >= threshold)[0]
    sorted_indices = indices[np.argsort(similarities[indices])[::-1]]
    return df.iloc[sorted_indices][['Product Id', 'Name', 'Category', 'SubCategory', 'Sale Price']]

In [25]:

user_input = "camphor aggarbatti"
results = recommend_products(user_input)
print("\n📦 Recommended Products:\n", results.to_string(index=False))


📦 Recommended Products:
  Product Id                         Name      Category SubCategory  Sale Price
       9570 Pratishtha Camphor Agarbatti Pooja Samagri   Agarbatti        25.0
       9041   Pratishtha Woods Agarbatti Pooja Samagri   Agarbatti        10.0
       9569   Pratishtha Mogra Agarbatti Pooja Samagri   Agarbatti         5.0
       9044  Pratishtha Sandal Agarbatti Pooja Samagri   Agarbatti         5.0
